### Reformat Enron Dataset as train and test
##### 10% test, 90% train

In [1]:
import shutil
import os
import re
from random import randint

def copyf(source, destination, depth=None):
    if not depth:
        depth = ""
    for file_or_dir in os.listdir(os.path.join(source + depth)):
        if os.path.isfile(os.path.join(source + depth, file_or_dir)):
#            print("cp " + os.path.join(source + depth, file_or_dir) + " " + destination)
            buf = depth.split("/")[-2:]
            fn = re.sub('[^0-9a-zA-Z]+', '_', "_".join(buf)) + '_' + file_or_dir + 'txt'
            if randint(0,99) > 85:
                if randint(0,99) > 85:
                    dest = os.path.join(destination, "test")
                else:
                    dest = os.path.join(destination, "train")                                                    
                if buf[1] == "sent":
                    shutil.copyfile(os.path.join(source + depth, file_or_dir), os.path.join(dest, fn))
        else:
            copyf('', destination, os.path.join(source + depth, file_or_dir))

In [2]:
source = "/home/wk/myProjects/data/maildir"
destination = "/home/wk/myProjects/data/Enron"
#copyf(source, destination, depth=None)

In [3]:
PATH='/home/wk/myProjects/data/Enron/'

TRN_PATH = 'train/'
VAL_PATH = 'test/'
TRN = f'{PATH}{TRN_PATH}'
VAL = f'{PATH}{VAL_PATH}'

%ls {PATH}

Enron_name_lookup.csv  oper/  TagSamples.ipynb  tmp/
models/                tag/   test/             train/


In [4]:
fname = !ls {TRN}
#trn_files = !dir /w {TRN}
fname[7:17]

['allen_p_sent_120.txt',
 'allen_p_sent_125.txt',
 'allen_p_sent_134.txt',
 'allen_p_sent_137.txt',
 'allen_p_sent_142.txt',
 'allen_p_sent_156.txt',
 'allen_p_sent_159.txt',
 'allen_p_sent_15.txt',
 'allen_p_sent_161.txt',
 'allen_p_sent_164.txt']

#### Search and Replace

In [5]:
def tag_by_meta_data(ss):
    e = ss.replace("\t", "").split("\n")
    d = [ee.split(':') for ee in e]
    tdoc = ss
    ToList = []
    iscapture = False
    for e in d[:30]:
        if e[0] == 'Message-ID':
            if e[1].strip() != "":
                tdoc = tdoc.replace(e[1].strip(), " @@MES_ID@@ ")
            continue
        if e[0] == 'From':
            if e[1].strip() != "":
                tdoc = tdoc.replace(e[1].strip(), " @@SNDR_EMAIL@@ ") 
            continue
        if e[0] == 'To':
            ToList = [a.strip() for a in e[1:]]
            iscapture = True
            continue
        if e[0] == 'Subject':
            iscapture = False
            i = 0
            for g in ToList:
                i = i + 1
                if g.strip() != "":
                    tdoc = tdoc.replace(g.strip(), " @@RECR_EMAIL" + str(i) + "@@ ")       
            continue
        if iscapture:
            ToList = ToList + [a.strip() for a in e]     
        if e[0] == 'X-From':        
            gg = e[1].strip().split(" ")
            if len(gg) == 1:
                if e[1].strip() != "":
                    tdoc = tdoc.replace(e[1].strip(), " @SNDR_FN@@ ")
            if len(gg) == 2:
                if e[1].strip() != "":
                    tdoc = tdoc.replace(e[1].strip(), " @@SNDR_FN@@ @@SNDR_LN@@ ")
            if len(gg) > 2:
                if e[1].strip() != "":
                    tdoc = tdoc.replace(e[1].strip(), " @@SNDR_FN@@ @@SNDR_MN@@ @@SNDR_LN@@ ")                    
            if gg[0].strip() != "":
                tdoc = tdoc.replace(gg[0].strip(), " @@SNDR_FN@@")
            if len(gg) > 1:
                if gg[len(gg)-1].strip() != "":                
                    tdoc = tdoc.replace(gg[len(gg)-1].strip(), " @@SNDR_LN@@ ")

        if e[0] == 'X-To':
            kk = e[1].strip().split(",") 
            i = 0
            for kkk in kk:
                i = i + 1
                kkkk = kkk.strip().split(" ")            
                if len(kkkk) == 1:
                    if kkk.strip() != "":
                        tdoc = tdoc.replace(kkk.strip(), " @@RCVR_" + str(i) + "FN@@ ")
                if len(kkkk) == 2:
                    if kkk.strip() != "":
                        tdoc = tdoc.replace(kkk.strip(), " @@RCVR_" + str(i) + "FN@@ @@RCVR_" + str(i) + "LN@@ ")
                if len(kkkk) > 2:
                    if kkk.strip() != "":
                        tdoc = tdoc.replace(kkk.strip(), " @@RCVR_" + str(i) + "FN@@ @@RCVR_" + str(i) + "MN@@ @@RCVR_" + str(i) + "LN@@ ")
                if kkkk[0].strip() != "":
                    tdoc = tdoc.replace(kkkk[0].strip(), " @@RCVR_" + str(i) + "FN@@ ")
                if len(kkkk) > 1:
                    if kkkk[len(kkkk)-1].strip() != "":
                        tdoc = tdoc.replace(kkkk[len(kkkk)-1].strip(), " @@RCVR_" + str(i) + "LN@@ ")
    return(tdoc)



In [6]:
import re
def search_and_replace_regex_in_file(fn_in, fn_out):
    # Read in the file
    with open(fn_in, 'r', errors="ignore") as file :
        line = file.read()
        line = tag_by_meta_data(line)
        line = re.sub(
                   r"""(<\d*.\d*.\w*.*@*>)""", 
                   " @@othr_mailsvr@@ ", 
                   line
               )        
        line = re.sub(
                   r"""(?:[a-z0-9!#$%&'*+/=?^_`{|}~-]+(?:\.[a-z0-9!#$%&'*+/=?^_`{|}~-]+)*|"(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21\x23-\x5b\x5d-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])*")@(?:(?:[a-z0-9](?:[a-z0-9-]*[a-z0-9])?\.)+[a-z0-9](?:[a-z0-9-]*[a-z0-9])?|\[(?:(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)\.){3}(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?|[a-z0-9-]*[a-z0-9]:(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21-\x5a\x53-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])+)\])""", 
                   " @@othr_em@@ ", 
                   line 
               )
        line = re.sub(
                   r"""(\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}|\(\d{3}\)\s*\d{3}[-\.\s]??\d{4}|\d{3}[-\.\s]??\d{4})""", 
                   " @@othr_ph@@ ", 
                   line
               )
        line = re.sub(
                   r"""(?i)\b((?:[a-z][\w-]+:(?:/{1,3}|[a-z0-9%])|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))""", 
                   " @@othr_ws@@ ", 
                   line
               )
        line = re.sub(
                   r"""\d{1,2}[/-]\d{1,2}[/-]\d{2,4}""", 
                   " @@othr_dt@@ ", 
                   line
               )
        line = re.sub(
                   r"""\d{1,} (?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]* \d{4}""", 
                   " @@othr_dt@@ ", 
                   line
               )
    with open(fn_out, 'w') as file:
        file.write(line)

In [7]:
dir_in = "/home/wk/myProjects/data/Enron/train/"
dir_out = "/home/wk/myProjects/data/Enron/tag/train/"

#### Replace with tag for training set

In [8]:
fname = !ls {dir_in}
fname[7:17]

['allen_p_sent_120.txt',
 'allen_p_sent_125.txt',
 'allen_p_sent_134.txt',
 'allen_p_sent_137.txt',
 'allen_p_sent_142.txt',
 'allen_p_sent_156.txt',
 'allen_p_sent_159.txt',
 'allen_p_sent_15.txt',
 'allen_p_sent_161.txt',
 'allen_p_sent_164.txt']

In [9]:
for fn in fname:  
    search_and_replace_regex_in_file(dir_in + fn, dir_out + fn)

In [10]:
dir_in = "/home/wk/myProjects/data/Enron/test/"
dir_out = "/home/wk/myProjects/data/Enron/tag/test/"

In [11]:
fname = !ls {dir_in}
fname[7:17]

['allen_p_sent_439.txt',
 'allen_p_sent_500.txt',
 'allen_p_sent_54.txt',
 'allen_p_sent_7.txt',
 'allen_p_sent_96.txt',
 'arnold_j_sent_100.txt',
 'arnold_j_sent_104.txt',
 'arnold_j_sent_113.txt',
 'arnold_j_sent_119.txt',
 'arnold_j_sent_120.txt']

In [12]:
for fn in fname:  
    search_and_replace_regex_in_file(dir_in + fn, dir_out + fn)

In [13]:
ss = r'''
Message-ID: <32498952.1075851920969.JavaMail.evans@thyme>
Date: Fri, 18 May 2001 05:38:00 -0700 (PDT)
From: larry.campbell@enron.com
To: jfields@argentinc.com
Subject: Midland Region tank spreadsheet
Mime-Version: 1.0
Content-Type: text/plain; charset=us-ascii
Content-Transfer-Encoding: 7bit
X-From: Larry Campbell
X-To: jfields@argentinc.com
X-cc: 
X-bcc: 
X-Folder: \Larry_Campbell_Nov2001_1\Notes Folders\Sent
X-Origin: CAMPBELL-L
X-FileName: lcampbe.nsf

---------------------- Forwarded by Larry Campbell/ET&S/Enron on 05/18/2001 
11:31 AM ---------------------------


Wade Jordan
05/15/2001 03:58 PM
To: Larry Campbell/ET&S/Enron@ENRON
cc: Bob Bandel/ET&S/Enron@ENRON 

Subject: Midland Region tank spreadsheet
'''
tag_by_meta_data(ss)

'\nMessage-ID:  @@MES_ID@@ \nDate: Fri, 18 May 2001 05:38:00 -0700 (PDT)\nFrom:  @@SNDR_EMAIL@@ \nTo:  @@RECR_EMAIL1@@ \nSubject: Midland Region tank spreadsheet\nMime-Version: 1.0\nContent-Type: text/plain; charset=us-ascii\nContent-Transfer-Encoding: 7bit\nX-From:  @@SNDR_FN@@ @@SNDR_LN@@ \nX-To:  @@RECR_EMAIL1@@ \nX- @@RECR_EMAIL2@@ : \nX-b @@RECR_EMAIL2@@ : \nX-Folder: \\ @@SNDR_FN@@_ @@SNDR_LN@@ _Nov2001_1\\Notes Folders\\Sent\nX-Origin: CAMPBELL-L\nX-FileName: lcampbe.nsf\n\n---------------------- Forwarded by  @@SNDR_FN@@ @@SNDR_LN@@ /ET&S/Enron on 05/18/2001 \n11:31 AM ---------------------------\n\n\nWade Jordan\n05/15/2001 03:58 PM\nTo:  @@SNDR_FN@@ @@SNDR_LN@@ /ET&S/Enron@ENRON\n @@RECR_EMAIL2@@ :  @@RECR_EMAIL3@@  \n\nSubject: Midland Region tank spreadsheet\n'